In [ ]:
###--- Tabula Table Extraction Process ---###
import os
import re
import pandas as pd
import tabula

#Pandas options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

PATH = os.getcwd()
FILE = '5-mpo-sm21-iran-irn-kcm.pdf'
FULL_PATH = os.path.join(PATH, 'Source_Files', FILE)

#Read PDF and extract tables
tables = tabula.read_pdf(FULL_PATH, pages="all")

#Separate relevant table from others
data = tables[2]
data.fillna(0, inplace=True) #Replace all NaN values

#Strip relevant column headers from dataframe
raw_head = data.iloc[0,:].tolist()
rt_head = []
for i in [re.findall('\d+/\d+', i) for i in raw_head[2].split()]:
    if len(i) > 0:
        rt_head.extend(i)
    else:
        pass
rh2 = raw_head[0:2]
rh3 = rh2+rt_head
rh2 = raw_head[0:2]
rh3 = ['Category']+rh2+rt_head

#Strip category names from DataFrame rows
col_1 = []
for i in range(1,len(data)):
    col_1.append(' '.join(re.findall('[A-z]+', data.iloc[i, 0])))
    
#Strip numbers from first column
col_2 = []
for i in range(1,len(data)):
    if len(re.findall('-\d+\.\d+', data.iloc[i, 0])) > 0:
        col_2.append(re.findall('-\d+\.\d+', data.iloc[i, 0]))
    elif len(re.findall('\d+\.\d+', data.iloc[i, 0])) > 0:
        col_2.append(re.findall('\d+\.\d+', data.iloc[i, 0]))
    else:
        col_2.append('0')

#Split numbers from string in third column
cols_3_6 = []
for row in data.iloc[1:, 2]:
    if len(str(row).split()) == 4:
        cols_3_6.append(str(row).split())
    else:
        cols_3_6.append(str(row).split()*4)

#Convert all lists back into DataFrames
df_1 = pd.DataFrame(col_1)
df_2 = pd.DataFrame(col_2)
df_3 = data.iloc[1:, [1]]
df_3.reset_index(inplace=True, drop=True) #Necessary due to slicing of previous DataFrame
df_4 = pd.DataFrame(cols_3_6)

#Concatenate all new DataFrames together
df_all = pd.concat([df_1, df_2, df_3, df_4], axis=1)

#Replace junk column names with legit names
df_all.columns = rh3

#df_all.to_csv('Iran_Economy_2018-2024.csv', index=False, sep='\t', encoding='utf-8')

print(f'Tables: {len(tables)}') #DEBUG